In [62]:
# -*- encoding:urf-8 -*-
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Handle table like and matrices
import pandas as pd
import numpy as np

# Modeling Helper
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
import missingno as msno

# Configure visualization
%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 8,6

train_path = '../input/train.csv'
test_path = '../input/test.csv'
hol_path = '../input/holiday.csv'
cache = '../input/cache/'
out_path = '../output/'
train = pd.read_csv(train_path,encoding='gbk',engine='python')
test = pd.read_csv(test_path)
holiday = pd.read_csv(hol_path)

In [63]:
train.rename(columns = {'BigCode':'parClass','MidCode':'Class'},inplace = True)
# train

### 重新整合train集

In [64]:
train.loc[:,'saleCount'] = 1
coord_class = train.groupby(['Class','SaleDate'],as_index=False)['saleCount'].sum()
coord_parClass = train.groupby(['parClass','SaleDate'],as_index=False)['saleCount'].sum()
coord_parClass.rename(columns = {'parClass':'Class'},inplace=True)
train_new = pd.concat([coord_class,coord_parClass],axis = 0)
train_new.loc[:,'parClass'] = train_new.Class.map(lambda x: str(x)[:2])
train_new.loc[:,'parClass'] = train_new.parClass.map(lambda x: int(x))
# train_new

### 节假日特征

In [36]:
train = pd.merge(train, holiday, on = 'SaleDate',how = 'left')
train_new = pd.merge(train_new, holiday, on = 'SaleDate',how = 'left')
test = pd.merge(test, holiday, on = 'SaleDate',how = 'left')

### 时间特征

In [37]:
def timeHandle(s):
    s = str(s)
    s = [s[:4],s[4:6],s[6:]]
    return '-'.join(s)

In [38]:
train.SaleDate = train.SaleDate.map(lambda x: timeHandle(x))
train.SaleDate = pd.to_datetime(train.SaleDate)
train.loc[:,'month'] = train.SaleDate.dt.month
train.loc[:,'dayOfWeek'] = train.SaleDate.dt.dayofweek
train.loc[:,'dayOfYear'] = train.SaleDate.dt.dayofyear
train.loc[:,'weekOfYear'] = train.SaleDate.dt.weekofyear

train_new.SaleDate = train_new.SaleDate.map(lambda x: timeHandle(x))
train_new.SaleDate = pd.to_datetime(train_new.SaleDate)
train_new.loc[:,'month'] = train_new.SaleDate.dt.month
train_new.loc[:,'dayOfWeek'] = train_new.SaleDate.dt.dayofweek
train_new.loc[:,'dayOfYear'] = train_new.SaleDate.dt.dayofyear
train_new.loc[:,'weekOfYear'] = train_new.SaleDate.dt.weekofyear

test.SaleDate = test.SaleDate.map(lambda x: timeHandle(x))
test.SaleDate = pd.to_datetime(test.SaleDate)
test.loc[:,'month'] = test.SaleDate.dt.month
test.loc[:,'dayOfWeek'] = test.SaleDate.dt.dayofweek
test.loc[:,'dayOfYear'] = test.SaleDate.dt.dayofyear
test.loc[:,'weekOfYear'] = test.SaleDate.dt.weekofyear

### 商品特征

#### 商品类别

In [39]:
cumDict = {u'一般商品':0.6089,u'生鲜':0.3782,'联营商品':0.0129}
train.CumType = train.CumType.map(cumDict)

In [40]:
midClassSet = set(train.Class)
bigClassSet = set(train.parClass)
midClassDict = {}
bigClassDict = {}
classDict = {}
for eachMid in midClassSet:
    coord = train[train.Class == eachMid].groupby('CumType')['Class'].count()
    sum = 0 
    for i in range(len(coord)):
        sum += coord.index[i] *  coord.values[i]
    rate = round(sum / (1.0 * len(coord) + 1),2) ##修正
    midClassDict[eachMid] = rate
    classDict[eachMid] = rate
for eachBig in bigClassSet:
    coord = train[train.parClass == eachBig].groupby('CumType')['Class'].count()
    sum = 0 
    for i in range(len(coord)):
        sum += coord.index[i] *  coord.values[i]
    rate = round(sum / (1.0 * len(coord) + 1),2) 
    bigClassDict[eachBig] = rate
    classDict[eachBig] = rate
train.loc[:,'cumType'] = train.Class.map(midClassDict)
train.loc[:,'parCumType'] = train.parClass.map(bigClassDict)

#最开始就需要改名
test.rename(columns={'Code':'Class'},inplace = True)
test.loc[:,'parClass'] = test.Class.map(lambda x: str(x)[:2])
test.loc[:,'parClass'] = test.parClass.map(lambda x: int(x))
test.loc[:,'cumType'] = test.Class.map(classDict)
test.loc[:,'parCumType'] = test.parClass.map(bigClassDict)

# 测试集中类cumType的缺失值处理,使用临近中类的type值
test = test.fillna('pad')

#### 商品热门指数

In [41]:
hotIndexDict = {}
parHotIndexDict = {}
totHotIndexDict = {}
totSaleCount = train.shape[0]
for eachMid in midClassSet:
    rate = round(train[train.Class == eachMid].shape[0] / (1.0 * totSaleCount),5)
    hotIndexDict[eachMid] = rate
    totHotIndexDict[eachMid] = rate
for eachBig in bigClassSet:
    rate = round(train[train.parClass == eachBig].shape[0] / (1.0 * totSaleCount),5)
    parHotIndexDict[eachBig] = rate
    totHotIndexDict[eachBig] = rate
    
train.loc[:,'hotIndex'] = train.Class.map(hotIndexDict)
train.loc[:,'parHotIndex'] = train.parClass.map(parHotIndexDict)

test.loc[:,'hotIndex'] = train.Class.map(totHotIndexDict)
test.loc[:,'parHotIndex'] = train.parClass.map(parHotIndexDict)
test

,Class,SaleDate,SaleNum,holidayCluster,disHoliday,month,dayOfWeek,dayOfYear,weekOfYear,parClass,cumType,parCumType,hotIndex,parHotIndex
0,1001,2015-05-01,1,3,1.0,5,4,121,18,10,175.86,157.19,0.23107,0.33148
1,1001,2015-05-02,1,3,1.0,5,5,122,18,10,175.86,157.19,0.01721,0.13720
2,1001,2015-05-03,1,3,1.0,5,6,123,18,10,175.86,157.19,0.02758,0.11165
3,1001,2015-05-04,1,1,0.5,5,0,124,19,10,175.86,157.19,0.00145,0.11165
4,1001,2015-05-05,1,1,0.3,5,1,125,19,10,175.86,157.19,0.02758,0.11165
5,1001,2015-05-06,1,1,0.3,5,2,126,19,10,175.86,157.19,0.01446,0.07166
6,1001,2015-05-07,1,1,0.0,5,3,127,19,10,175.86,157.19,0.23107,0.33148
7,1001,2015-05-08,1,1,0.0,5,4,128,19,10,175.86,157.19,0.00677,0.13720
8,1001,2015-05-09,1,2,0.0,5,5,129,19,10,175.86,157.19,0.01042,0.02076
9,1001,2015-05-10,1,2,0.0,5,6,130,19,10,175.86,157.19,0.03410,0.20668


#### 商品工作日/非工作日销量统计值 - 用train_new表

In [42]:
# train_new

In [43]:
train_wk = train_new[train_new.holidayCluster == 1]
train_hol = train_new[train_new.holidayCluster != 1]

coord = train_wk.groupby('Class',as_index = False)['saleCount'].agg({'wkDaySaleCount_median':'median'})
train_new = pd.merge(train_new, coord, on = 'Class', how='left')
test = pd.merge(test, coord, on = 'Class', how='left')
coord = train_wk.groupby('Class',as_index = False)['saleCount'].agg({'wkDaySaleCount_mean':'mean'})
train_new = pd.merge(train_new, coord, on = 'Class', how='left')
test = pd.merge(test, coord, on = 'Class', how='left')
coord = train_wk.groupby('Class',as_index = False)['saleCount'].agg({'wkDaySaleCount_max':'max'})
train_new = pd.merge(train_new, coord, on = 'Class', how='left')
test = pd.merge(test, coord, on = 'Class', how='left')
coord = train_wk.groupby('Class',as_index = False)['saleCount'].agg({'wkDaySaleCount_min':'min'})
train_new = pd.merge(train_new, coord, on = 'Class', how='left')
test = pd.merge(test, coord, on = 'Class', how='left')

coord = train_hol.groupby('Class',as_index = False)['saleCount'].agg({'holDaySaleCount_median':'median'})
train_new = pd.merge(train_new, coord, on = 'Class', how='left')
test = pd.merge(test, coord, on = 'Class', how='left')
coord = train_hol.groupby('Class',as_index = False)['saleCount'].agg({'holDaySaleCount_mean':'mean'})
train_new = pd.merge(train_new, coord, on = 'Class', how='left')
test = pd.merge(test, coord, on = 'Class', how='left')
coord = train_hol.groupby('Class',as_index = False)['saleCount'].agg({'holDaySaleCount_max':'max'})
train_new = pd.merge(train_new, coord, on = 'Class', how='left')
test = pd.merge(test, coord, on = 'Class', how='left')
coord = train_hol.groupby('Class',as_index = False)['saleCount'].agg({'holDaySaleCount_min':'min'})
train_new = pd.merge(train_new, coord, on = 'Class', how='left')
test = pd.merge(test, coord, on = 'Class', how='left')

coord = train_hol.groupby('Class',as_index=False)['saleCount'].agg({'holSaleCount':'count'})
train_new = pd.merge(train_new, coord, on = 'Class', how='left')
coord = train_wk.groupby('Class',as_index=False)['saleCount'].agg({'wkSaleCount':'count'})
train_new = pd.merge(train_new, coord, on = 'Class', how='left')
train_new.loc[:,'wkHolRatio'] = train_new['wkSaleCount'] / (1.0 * train_new['holSaleCount'])

coord = train_new.groupby('Class',as_index=False)['wkHolRatio'].mean()
test = pd.merge(test, coord, on = 'Class', how='left')

del train_new['wkSaleCount'],train_new['holSaleCount']
del train_wk,train_hol

# train_new

#### 价格特征

In [44]:
coord_class = train.groupby('Class',as_index = False)['UnitPrice'].agg({'price_mean':'mean'})
coord_par_class = train.groupby('parClass',as_index = False)['UnitPrice'].agg({'price_mean':'mean'})
coord_par_class.rename(columns = {'parClass':'Class'},inplace = True)
coord = pd.concat([coord_class, coord_par_class],axis = 0)
train_new = pd.merge(train_new, coord, on = 'Class', how = 'left')
test = pd.merge(test, coord, on = 'Class', how = 'left')

coord_class = train.groupby('Class',as_index = False)['UnitPrice'].agg({'price_median':'median'})
coord_par_class = train.groupby('parClass',as_index = False)['UnitPrice'].agg({'price_median':'median'})
coord_par_class.rename(columns = {'parClass':'Class'},inplace = True)
coord = pd.concat([coord_class, coord_par_class],axis = 0)
train_new = pd.merge(train_new, coord, on = 'Class', how = 'left')
test = pd.merge(test, coord, on = 'Class', how = 'left')

#### 促销特征

商品促销时销量与不促销时销量的比值 

In [45]:
coord_class_bonus_count = train[train['Coupon'] == 1].groupby('Class',as_index=False)['saleCount'].agg({'classBonusSaleCount':'count'})
coord_parclass_bonus_count = train[train['Coupon'] == 1].groupby('parClass',as_index=False)['saleCount'].agg({'classBonusSaleCount':'count'})
coord_parclass_bonus_count.rename(columns={'parClass':'Class'},inplace = True)
coord = pd.concat([coord_class_bonus_count,coord_parclass_bonus_count],axis=0)
train_new = pd.merge(train_new, coord, on = 'Class', how = 'left')
test = pd.merge(test, coord, on = 'Class', how = 'left')
train_new['classBonusSaleCount'] = train_new['classBonusSaleCount'].fillna(0)
test['classBonusSaleCount'] = test['classBonusSaleCount'].fillna(0)

coord_class_notbonus_count = train[train['Coupon'] == 0].groupby('Class',as_index=False)['saleCount'].agg({'classNotBonusSaleCount':'count'})
coord_parclass_notbonus_count = train[train['Coupon'] == 0].groupby('parClass',as_index=False)['saleCount'].agg({'classNotBonusSaleCount':'count'})
coord_parclass_notbonus_count.rename(columns={'parClass':'Class'},inplace = True)
coord = pd.concat([coord_class_notbonus_count,coord_parclass_notbonus_count],axis=0)
train_new = pd.merge(train_new, coord, on = 'Class', how = 'left')
test = pd.merge(test, coord, on = 'Class', how = 'left')
train_new['classNotBonusSaleCount'] = train_new['classNotBonusSaleCount'].fillna(1)
test['classNotBonusSaleCount'] = test['classNotBonusSaleCount'].fillna(1)

# 计算促销与非促销的比值
train_new.loc[:,'bonusRatio'] = np.round(train_new['classBonusSaleCount'] / (1.0 * train_new['classNotBonusSaleCount']),4)
del train_new['classBonusSaleCount'],train_new['classNotBonusSaleCount']
test.loc[:,'bonusRatio'] = np.round(test['classBonusSaleCount'] / (1.0 * test['classNotBonusSaleCount']),4)
del test['classBonusSaleCount'],test['classNotBonusSaleCount']

商品节假日时促销销量与不促销销量的比值

In [46]:
train_wk = train[train.holidayCluster == 1]
train_hol = train[train.holidayCluster != 1]

coord_class_bonus_count = train_hol[train_hol['Coupon'] == 1].groupby('Class',as_index=False)['saleCount'].agg({'classBonusSaleCount':'count'})
coord_parclass_bonus_count = train_hol[train_hol['Coupon'] == 1].groupby('parClass',as_index=False)['saleCount'].agg({'classBonusSaleCount':'count'})
coord_parclass_bonus_count.rename(columns={'parClass':'Class'},inplace = True)
coord = pd.concat([coord_class_bonus_count,coord_parclass_bonus_count],axis=0)
train_new = pd.merge(train_new, coord, on = 'Class', how = 'left')
test = pd.merge(test, coord, on = 'Class', how = 'left')
train_new['classBonusSaleCount'] = train_new['classBonusSaleCount'].fillna(0)
test['classBonusSaleCount'] = test['classBonusSaleCount'].fillna(0)

coord_class_notbonus_count = train_hol[train_hol['Coupon'] == 0].groupby('Class',as_index=False)['saleCount'].agg({'classNotBonusSaleCount':'count'})
coord_parclass_notbonus_count = train_hol[train_hol['Coupon'] == 0].groupby('parClass',as_index=False)['saleCount'].agg({'classNotBonusSaleCount':'count'})
coord_parclass_notbonus_count.rename(columns={'parClass':'Class'},inplace = True)
coord = pd.concat([coord_class_notbonus_count,coord_parclass_notbonus_count],axis=0)
train_new = pd.merge(train_new, coord, on = 'Class', how = 'left')
test = pd.merge(test, coord, on = 'Class', how = 'left')
train_new['classNotBonusSaleCount'] = train_new['classNotBonusSaleCount'].fillna(1)
test['classNotBonusSaleCount'] = test['classNotBonusSaleCount'].fillna(1)

# 计算促销与非促销的比值
train_new.loc[:,'bonusHolRatio'] = np.round(train_new['classBonusSaleCount'] / (1.0 * train_new['classNotBonusSaleCount']),4)
del train_new['classBonusSaleCount'],train_new['classNotBonusSaleCount']
test.loc[:,'bonusHolRatio'] = np.round(test['classBonusSaleCount'] / (1.0 * test['classNotBonusSaleCount']),4)
del test['classBonusSaleCount'],test['classNotBonusSaleCount']

商品非节假日时促销销量与不促销销量的比值

In [47]:
coord_class_bonus_count = train_wk[train_wk['Coupon'] == 1].groupby('Class',as_index=False)['saleCount'].agg({'classBonusSaleCount':'count'})
coord_parclass_bonus_count = train_wk[train_wk['Coupon'] == 1].groupby('parClass',as_index=False)['saleCount'].agg({'classBonusSaleCount':'count'})
coord_parclass_bonus_count.rename(columns={'parClass':'Class'},inplace = True)
coord = pd.concat([coord_class_bonus_count,coord_parclass_bonus_count],axis=0)
train_new = pd.merge(train_new, coord, on = 'Class', how = 'left')
test = pd.merge(test, coord, on = 'Class', how = 'left')
train_new['classBonusSaleCount'] = train_new['classBonusSaleCount'].fillna(0)
test['classBonusSaleCount'] = test['classBonusSaleCount'].fillna(0)

coord_class_notbonus_count = train_wk[train_wk['Coupon'] == 0].groupby('Class',as_index=False)['saleCount'].agg({'classNotBonusSaleCount':'count'})
coord_parclass_notbonus_count = train_wk[train_wk['Coupon'] == 0].groupby('parClass',as_index=False)['saleCount'].agg({'classNotBonusSaleCount':'count'})
coord_parclass_notbonus_count.rename(columns={'parClass':'Class'},inplace = True)
coord = pd.concat([coord_class_notbonus_count,coord_parclass_notbonus_count],axis=0)
train_new = pd.merge(train_new, coord, on = 'Class', how = 'left')
test = pd.merge(test, coord, on = 'Class', how = 'left')
train_new['classNotBonusSaleCount'] = train_new['classNotBonusSaleCount'].fillna(1)
test['classNotBonusSaleCount'] = test['classNotBonusSaleCount'].fillna(1)

# 计算促销与非促销的比值
train_new.loc[:,'bonusNotHolRatio'] = np.round(train_new['classBonusSaleCount'] / (1.0 * train_new['classNotBonusSaleCount']),4)
del train_new['classBonusSaleCount'],train_new['classNotBonusSaleCount']
test.loc[:,'bonusNotHolRatio'] = np.round(test['classBonusSaleCount'] / (1.0 * test['classNotBonusSaleCount']),4)
del test['classBonusSaleCount'],test['classNotBonusSaleCount']

del train_wk,train_hol

商品周几促销的比例

In [48]:
train_coupon = train[train.Coupon == 1]
coord = train_coupon.groupby(['Class','dayOfWeek'],as_index=False)['dayOfWeek'].agg({'dayOfWeekCount':'count'})
var = train_coupon.groupby(['Class'],as_index=False)['dayOfWeek'].agg({'classCouponCount':'count'})
coord = pd.merge(coord, var, on = 'Class',how='left' )
coord.loc[:,'bonusWeekProb'] = coord['dayOfWeekCount'] / np.round((1.0 * coord['classCouponCount']))
coord_c = coord.copy()

coord = train_coupon.groupby(['parClass','dayOfWeek'],as_index=False)['dayOfWeek'].agg({'dayOfWeekCount':'count'})
var = train_coupon.groupby(['parClass'],as_index=False)['dayOfWeek'].agg({'classCouponCount':'count'})
coord = pd.merge(coord, var, on = 'parClass',how='left' )
coord.loc[:,'bonusWeekProb'] = coord['dayOfWeekCount'] / np.round((1.0 * coord['classCouponCount']))
coord.rename(columns={'parClass':'Class'},inplace=True)
coord_pc = coord.copy()

coord = pd.concat([coord_c,coord_pc],axis=0)

train_new = pd.merge(train_new, coord[['Class','dayOfWeek','bonusWeekProb']], on = ['Class','dayOfWeek'],how='left')
train_new['bonusWeekProb'] = train_new['bonusWeekProb'].fillna(0)

test = pd.merge(test, coord[['Class','dayOfWeek','bonusWeekProb']], on = ['Class','dayOfWeek'],how='left')
test['bonusWeekProb'] = test['bonusWeekProb'].fillna(0)

## 测试集分割

In [50]:
test.shape

(4440, 29)

In [52]:
train_new.shape

(8998, 25)

In [53]:
train_new.Class.unique().size

193

In [54]:
test.Class.unique().size

148

In [61]:
train_date = pd.read_csv('../input/train_date.csv')
train_date_new = pd.merge(train_new, train_date, on = 'SaleDate',how = 'outer')
train_date_new

,Class,SaleDate,saleCount,parClass,holidayCluster,disHoliday,month,dayOfWeek,dayOfYear,weekOfYear,...,holDaySaleCount_mean,holDaySaleCount_max,holDaySaleCount_min,wkHolRatio,price_mean,price_median,bonusRatio,bonusHolRatio,bonusNotHolRatio,bonusWeekProb
0,1001.0,2015-01-01 00:00:00.000000000,3.0,10.0,3.0,1.0,1.0,3.0,1.0,1.0,...,5.631579,16.0,1.0,1.973684,29.609892,29.80,0.0000,0.0000,0.0000,0.000000
1,1004.0,2015-01-01 00:00:00.000000000,1.0,10.0,3.0,1.0,1.0,3.0,1.0,1.0,...,1.925926,4.0,1.0,2.259259,24.730942,19.60,0.0000,0.0000,0.0000,0.000000
2,1101.0,2015-01-01 00:00:00.000000000,1.0,11.0,3.0,1.0,1.0,3.0,1.0,1.0,...,1.333333,4.0,1.0,1.666667,28.485632,30.00,0.0000,0.0000,0.0000,0.000000
3,1103.0,2015-01-01 00:00:00.000000000,1.0,11.0,3.0,1.0,1.0,3.0,1.0,1.0,...,1.000000,1.0,1.0,2.500000,29.515152,30.00,0.0000,0.0000,0.0000,0.000000
4,1201.0,2015-01-01 00:00:00.000000000,64.0,12.0,3.0,1.0,1.0,3.0,1.0,1.0,...,74.512821,148.0,14.0,1.974359,6.806642,5.60,0.0000,0.0000,0.0000,0.000000
5,1202.0,2015-01-01 00:00:00.000000000,5.0,12.0,3.0,1.0,1.0,3.0,1.0,1.0,...,6.861111,16.0,1.0,2.138889,5.715249,2.96,0.0000,0.0000,0.0000,0.000000
6,1203.0,2015-01-01 00:00:00.000000000,33.0,12.0,3.0,1.0,1.0,3.0,1.0,1.0,...,23.871795,44.0,5.0,1.948718,14.199199,11.98,0.0000,0.0000,0.0000,0.000000
7,1205.0,2015-01-01 00:00:00.000000000,1.0,12.0,3.0,1.0,1.0,3.0,1.0,1.0,...,1.550000,4.0,1.0,2.650000,11.220126,2.00,0.0000,0.0000,0.0000,0.000000
8,1301.0,2015-01-01 00:00:00.000000000,2.0,13.0,3.0,1.0,1.0,3.0,1.0,1.0,...,2.484848,7.0,1.0,2.030303,21.051064,20.00,0.0000,0.0000,0.0000,0.000000
9,1302.0,2015-01-01 00:00:00.000000000,2.0,13.0,3.0,1.0,1.0,3.0,1.0,1.0,...,1.944444,5.0,1.0,2.277778,74.380000,78.00,0.0000,0.0000,0.0000,0.000000
